In [ ]:
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Model
from matplotlib import pyplot as plt

def define_discriminator():
    inputs = Input(shape=(  )) # discriminator input的維度必須等於generator output及函式的維度
    ''''
    請建立discriminator網路
    ''''
    outputs = ...  # discriminator output 的維度等於 1
    
    model = Model(inputs, outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def define_generator(latent_dim):
    inputs = Input(shape=(  )) # generator input 的維度必須等於latent_dim
    ''''
    請建立generator網路
    ''''
    outputs = ... # generator output 的維度必須等於函式的維度(2)
    
    model = Model(inputs, outputs)
    return model

def define_gan(generator, discriminator):
    #將discriminator的參數設成不可訓練
    discriminator.trainable = False
    
    #連接generator, discriminator
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

# 生成 n 個真實樣本和標籤
def generate_real_samples(n):
    # 生成 [-0.5, 0.5] 範圍內的輸入值
    X1 = rand(n) - 0.5
    # 生成輸出值 X^2
    X2 = X1 * X1
    
    X1 = X1.reshape(n, 1)
    X2 = X2.reshape(n, 1)
    X = hstack((X1, X2))
    
    # 生成真實標籤
    y = ones((n, 1))
    
    return X, y

# 生成latent_space中的點作為生成器的輸入
def generate_latent_points(latent_dim, n):
    # 在latent_space中生成點
    x_input = randn(latent_dim * n)
    
    x_input = x_input.reshape(n, latent_dim)
    return x_input

# 用生成器生成 n 個假樣本和類標籤
def generate_fake_samples(generator, latent_dim, n):
    # 在letent_space中生成點
    x_input = generate_latent_points(latent_dim, n)
    # 預測輸出值
    X = generator.predict(x_input)
    # 創建假標籤
    y = zeros((n, 1))
    return X, y

# 評估判別器並且繪製真假點
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
    # 準備真實樣本
    x_real, y_real = generate_real_samples(n)
    # 在真實樣本上評估判別器
    _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
    # 準備假樣本
    x_fake, y_fake = generate_fake_samples(generator, latent_dim, n)
    # 在假樣本上評估判別器
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
    # 總結判別器性能
    print(epoch, acc_real, acc_fake)
    # 繪製真假數據的散點圖
    plt.scatter(x_real[:, 0], x_real[:, 1], color='red')
    plt.scatter(x_fake[:, 0], x_fake[:, 1], color='blue')
    plt.show()
    
def train(g_model, d_model, gan_model, latent_dim, n_epochs=..., n_batch=..., n_eval=...):
    # 用一半的 batch 來訓練判別器
    half_batch = int(n_batch / 2)
    
    for i in range(n_epochs):
        #生成真假樣本
        x_real, y_real = generate_real_samples(half_batch)
        x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        
        for j in range(...):
            #先訓練discriminator
            d_model.train_on_batch( ... , ... )
            d_model.train_on_batch( ... , ... )
        
        for j in range(...):
            #生成隨機的值給generator input
            x_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = ...

            #訓練generator騙過discriminator
            gan_model.train_on_batch( ... , ... )
        
        if (i+1) % n_eval == 0:
            summarize_performance(i, g_model, d_model, latent_dim)
            
latent_dim = 5
# 建立判別器
discriminator = define_discriminator()
# 建立生成器
generator = define_generator(latent_dim)
# 建立生成對抗網絡
gan_model = define_gan(generator, discriminator)
# 訓練
train(generator, discriminator, gan_model, latent_dim)